# Writing Formatters for dotnet interactive

Dotnet interactive is great, you just have to define your types, bind values to a name...

In [1]:
#!fsharp
type Person =
    { Name: string
      Age: int }

let alice = { Name = "Alice"
              Age = 33 }

...and without a lot of fuss you can display them. If you don't bind an expression to a name or actively ignore it dotnet interactive will just assume, that you want the value to be printed. For F# records dotnet interactive will choose the form of a nice and tidy table.

In [1]:
#!fsharp
alice

Name,Age
Alice,33


The standard formatter also knows how to handle composed records sensibly. How sweet!

In [1]:
#!fsharp
type Car =
    { Make: string
      Owner: Person }

let alicesCar = { Make = "Toyota"; Owner = alice }

alicesCar

Make,Owner
Toyota,"{ Name = ""Alice"" Age = 33 }"


As you can imagine this also works for Plain Old CLR types.

In [1]:
#!fsharp
type Dog(name: string, isGoodBoy: bool) =
    member _.Name = name
    member _.IsGoodBoy = isGoodBoy

let henry = Dog("Henry", true)

henry

Name,IsGoodBoy
Henry,True


Composing normal CLR classes or structs don't offer the nice formatting used for F# records, though.

In [1]:
#!fsharp
type DogHotel(name: string, inhabitants: Dog list) =
    member _.Name = name
    member _.Inhabitant = inhabitants

let hiltonForDogs = DogHotel("Hilton For Dogs", [ henry ])

hiltonForDogs

Name,Inhabitant
Hilton For Dogs,[ FSI_0007+Dog ]


Discriminated Unions are a bit of a different story because they offer a lot of room for composition. A simple union type containing only labels is pretty straight forward to print.

In [1]:
#!fsharp
type Fruit =
    | Orange
    | Banana
    | Apple

Apple

Apple

It doesn't have to (and it usually doesn't) stay that simple. Discriminated unions allow sticking data to labels. You can even give expressive names to the different data cells. Dotnet interactive is pretty flexible regarding the way it formats discriminated unions.

In [1]:
#!fsharp
type GiftBasket =
    | EmptyBasket
    | FruitBasket of Fruits: Fruit list
    | SpoiledFruitBasket of Fruit list

let aNiceGiftBasket = FruitBasket [ Orange; Orange; Banana; Banana; Banana; Apple ]
let aNotVeryNiceGiftBasket = SpoiledFruitBasket [ Orange; Orange; Banana; Banana; Banana; Apple ]

display aNiceGiftBasket
display aNotVeryNiceGiftBasket
display [ aNiceGiftBasket; aNotVeryNiceGiftBasket ]

Fruits
"[ Orange, Orange, Banana, Banana, Banana, Apple ]"


Item
"[ Orange, Orange, Banana, Banana, Banana, Apple ]"


index,type,Fruits,Item
0,FSI_0010+GiftBasket+FruitBasket,"[ Orange, Orange, Banana, Banana, Banana, Apple ]",
1,FSI_0010+GiftBasket+SpoiledFruitBasket,,"[ Orange, Orange, Banana, Banana, Banana, Apple ]"


One thing I have mixed feelings about is the formatting of single union cases with data. I personally like to see the case label. It just tells me a tiny bit more about the data I have in front of me. Sure, I can read it out of the code cell's context, but I like it to be printed as well.

In [1]:
#!fsharp
printfn "%A" aNiceGiftBasket

FruitBasket [Orange; Orange; Banana; Banana; Banana; Apple]

Fortunately dotnet interactive allows us to register custom formatters. In the case of the `GiftBasket` type I want to make sure, that I don't mix up `FruitBasket`s and `SpoiledFruitBasket`s. Adding a formatter for a type is quite easy (event though it looks kind of odd in F#).

In [1]:
#!fsharp
module GiftBasketFormatter =

    open System.Text

    // Set `GiftBasket`'s preferred formatter to match it with our new formatter
    Formatter.SetPreferredMimeTypeFor(typeof<GiftBasket> ,"text/plain")

    // Register a formatter for `GiftBasket` that uses the standard F# string format for discriminated unions
    Formatter.Register<GiftBasket>(Func<FormatContext, GiftBasket, TextWriter, bool>(fun context basket writer ->
        let formatted = sprintf "%A" basket
        writer.Write(formatted)
        
        true), "text/plain", true)

So far so good! Now we see what kind of gift basket we have in front of us. Of course this example is a bit contrived. There are of course better examples.

In [1]:
#!fsharp
display aNiceGiftBasket
display aNotVeryNiceGiftBasket

FruitBasket [Orange; Orange; Banana; Banana; Banana; Apple]

SpoiledFruitBasket [Orange; Orange; Banana; Banana; Banana; Apple]

Let's load `Plotly.NET` the officially condoned way of creating Plotly charts in .NET.

In [1]:
#!fsharp
#r "nuget: Plotly.NET, 2.0.0-alpha3"

Installed package Plotly.NET version 2.0.0-alpha3

If you are like me you might not be the best at guessing distributions from means, standard deviations and other numerical cues. Personally, I'd rather look at the values in a chart and go from there. But first let's generate some data. As far as I know the standard .NET `Random` generator is uniformly distributed. So I'll go and roll a uniformly distributed dice for a thousand times. I'll be back in a sec.

In [1]:
#!fsharp
open System

let rollUniformDice (rnd: Random) =
    rnd.Next(1, 7)
    |> int

let rnd = Random(Seed = 1)

let diceRolls = List.init 1000 (fun _ -> rollUniformDice rnd)

A good way to learn more about the distribution is to take a quick glimpse at a histogram. `Plotly.NET` makes this pretty easy. But unfortunately dotnet interactive doesn't really know how to handle those type of charts. If we look into the API we'll see that all those common charting functions return `GenericChart` objects.

In [1]:
#!fsharp
open Plotly.NET

diceRolls
|> Chart.Histogram

Item1,Item2,Item3
Plotly.NET.Trace,Plotly.NET.Layout,Plotly.NET.Config


As we've seen above, regestering a formatter for a type is straight forward. Jupyter Notebooks are built on top of web technologies, so as long as we can somehow produce some combination of HTML, JavaScript and/or images we're good to go. Fortunately `Plotly.NET` makes it really easy to do exactly that.

In [1]:
#!fsharp
module PlotlyFormatter =
    open System.Text
    open GenericChart

    Formatter.Register<GenericChart.GenericChart>(Func<FormatContext, GenericChart.GenericChart, TextWriter, bool>(fun context chart writer ->
        let html = toChartHTML chart
        writer.Write(html)
        true), HtmlFormatter.MimeType)

Now that's better! As expected for a uniformly distributed random generator we get a rather uniformly distributed histogram. Quite obvious when you see it in a chart, right?

In [1]:
#!fsharp
diceRolls
|> Chart.Histogram

<!-- Plotly chart will be drawn inside this DIV -->

So that was fun. You know what would even be more fun? Plotting other distributions! .NET doesn't really come with other random generators but we can build one based on a normal (or Gaussian) distribution ourselves using the [Box-Muller transform](https://en.wikipedia.org/wiki/Box%E2%80%93Muller_transform). All we need to implement this is a uniform random generator and a bit of (googleable) math. When we plot a histogram with 1000 samples we'll see the good old Gaussian bell curve. Neat!

In [1]:
#!fsharp
let normalRandom (rnd: Random) =
    // mean of standardized normal distribution
    let mu = 0.
    // standard deviation of normalized standard distribution
    let sigma = 1.

    let u1 = 1. - rnd.NextDouble()
    let u2 = 1. - rnd.NextDouble()
    let rndStdNormal =
        Math.Sqrt(-2. * Math.Log(u1)) * Math.Sin(2. * Math.PI * u2)

    mu + sigma * rndStdNormal

let rnd = Random(Seed = 1)

List.init 1000 (fun _ -> normalRandom rnd)
|> Chart.Histogram

<!-- Plotly chart will be drawn inside this DIV -->

Of course we don't really want to write computational numerics code ourselves in order to play around with numbers and charts for every kind of distribution. The .NET ecosystem is big and rich and it `MathNet.Numerics` package offers a treassure trove of mathematical playthings.

In [1]:
#!fsharp
#r "nuget: MathNet.Numerics"

Installed package MathNet.Numerics version 4.12.0

The next sample also takes 1000 samples from a normal distribution created using `MathNet`. For good meassure we make the most of our notebook and display some descriptive statistics right over the histogram. Charts are nice but in combination with a couple of succinct metrics it is just much more informative.

In [1]:
#!fsharp
open MathNet.Numerics.Distributions
open MathNet.Numerics.Random
open MathNet.Numerics.Statistics

let normal = Normal.WithMeanStdDev(0., 1., Random(Seed = 1))

let samples =
    normal.Samples()
    |> Seq.take 1000
    |> Seq.toList

let descriptiveStatistics = DescriptiveStatistics(samples)
display descriptiveStatistics

samples
|> Chart.Histogram

Count,Mean,Variance,StandardDeviation,Skewness,Kurtosis,Maximum,Minimum
1000,0.005356734049714328,0.9722233287545327,0.9860138582974037,0.10537912454150053,-0.2600928624047911,3.0928720069557665,-2.631058394828028


<!-- Plotly chart will be drawn inside this DIV -->